In [0]:
import numpy as np
import json
import os
from keras.models import Sequential
from keras.layers import Dense, Input, Dropout, LSTM, Activation, Convolution1D, MaxPooling2D, Flatten
from keras.utils import np_utils
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint

In [0]:
!mkdir data

In [0]:
!ls ./data

In [12]:
!gsutil -m cp gs://quickdraw_dataset/full/raw/* ./data

Copying gs://quickdraw_dataset/full/raw/The Eiffel Tower.ndjson...
Copying gs://quickdraw_dataset/full/raw/The Great Wall of China.ndjson...
Copying gs://quickdraw_dataset/full/raw/The Mona Lisa.ndjson...
Copying gs://quickdraw_dataset/full/raw/aircraft carrier.ndjson...
Copying gs://quickdraw_dataset/full/raw/airplane.ndjson...
Copying gs://quickdraw_dataset/full/raw/ambulance.ndjson...
Copying gs://quickdraw_dataset/full/raw/alarm clock.ndjson...
Copying gs://quickdraw_dataset/full/raw/angel.ndjson...
Copying gs://quickdraw_dataset/full/raw/animal migration.ndjson...
Copying gs://quickdraw_dataset/full/raw/ant.ndjson...
Copying gs://quickdraw_dataset/full/raw/anvil.ndjson...
Copying gs://quickdraw_dataset/full/raw/apple.ndjson...
Copying gs://quickdraw_dataset/full/raw/arm.ndjson...
Copying gs://quickdraw_dataset/full/raw/asparagus.ndjson...
Copying gs://quickdraw_dataset/full/raw/axe.ndjson...
Copying gs://quickdraw_dataset/full/raw/backpack.ndjson...
Copying gs://quickdraw_dataset/

In [13]:
!ls ./data

'aircraft carrier.ndjson'   couch.ndjson
 airplane.ndjson	    cow.ndjson
'alarm clock.ndjson'	    crab.ndjson
 ambulance.ndjson	    crayon.ndjson
 angel.ndjson		    crocodile.ndjson
'animal migration.ndjson'   crown.ndjson
 ant.ndjson		   'cruise ship.ndjson'
 anvil.ndjson		    cup.ndjson
 apple.ndjson		    diamond.ndjson
 arm.ndjson		    dishwasher.ndjson
 asparagus.ndjson	   'diving board.ndjson'
 axe.ndjson		    dog.ndjson
 backpack.ndjson	    dolphin.ndjson
 banana.ndjson		    donut.ndjson
 bandage.ndjson		    door.ndjson
 barn.ndjson		    dragon.ndjson
'baseball bat.ndjson'	    dresser.ndjson
 baseball.ndjson	    drill.ndjson
 basketball.ndjson	    drums.ndjson
 basket.ndjson		    duck.ndjson
 bathtub.ndjson		    dumbbell.ndjson
 bat.ndjson		    ear.ndjson
 beach.ndjson		    elbow.ndjson
 beard.ndjson		    elephant.ndjson
 bear.ndjson		    envelope.ndjson
 bed.ndjson		    eraser.ndjson
 bee.ndjson		    eyeglasses.ndjson
 belt.ndjson		    eye.ndjson
 bench.ndjson		    face.ndjson
 

In [0]:
def parse_line(ndjson_line):
  """Parse an ndjson line and return ink (as np array) and classname."""
  sample = json.loads(ndjson_line)
  class_name = sample["word"]
  if not class_name:
    print ("Empty classname")
    return None, None
  inkarray = sample["drawing"]
  stroke_lengths = [len(stroke[0]) for stroke in inkarray]
  total_points = sum(stroke_lengths)
  np_ink = np.zeros((total_points, 3), dtype=np.float32)
  current_t = 0
  if not inkarray:
    print("Empty inkarray")
    return None, None
  for stroke in inkarray:
    if len(stroke[0]) != len(stroke[1]):
      print("Inconsistent number of x and y coordinates.")
      return None, None
    for i in [0, 1]:
      np_ink[current_t:(current_t + len(stroke[0])), i] = stroke[i]
    current_t += len(stroke[0])
    np_ink[current_t - 1, 2] = 1  # stroke_end
  # Preprocessing.
  # 1. Size normalization.
  lower = np.min(np_ink[:, 0:2], axis=0)
  upper = np.max(np_ink[:, 0:2], axis=0)
  scale = upper - lower
  scale[scale == 0] = 1
  np_ink[:, 0:2] = (np_ink[:, 0:2] - lower) / scale
  # 2. Compute deltas.
  np_ink[1:, 0:2] -= np_ink[0:-1, 0:2]
  np_ink = np_ink[1:, :]
  return np_ink, class_name

In [0]:
alarm = './data/alarm clock.ndjson'
book = './data/book.ndjson'
campfire = './data/campfire.ndjson'
cloud = './data/cloud.ndjson'

In [0]:
data_x = np.empty((0,100,3), float)

In [17]:
ctr=0
with open(alarm) as f:
	for line in f:
		a,b = parse_line(line)
		if a.shape[0]>=100:
			ctr+=1
			print ctr
			if ctr>10000:
				break
			a = a[:100,:]
			a[-1,2] = 1.0
			a = a.reshape(1,100,3)
			data_x = np.append(data_x, a, axis=0)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [18]:
ctr=0
with open(book) as f:
	for line in f:
		a,b = parse_line(line)
		if a.shape[0]>=100:
			ctr+=1
			print ctr
			if ctr>10000:
				break
			a = a[:100,:]
			a[-1,2] = 1.0
			a = a.reshape(1,100,3)
			data_x = np.append(data_x, a, axis=0)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [19]:
ctr=0
with open(campfire) as f:
	for line in f:
		a,b = parse_line(line)
		if a.shape[0]>=100:
			ctr+=1
			print ctr
			if ctr>10000:
				break
			a = a[:100,:]
			a[-1,2] = 1.0
			a = a.reshape(1,100,3)
			data_x = np.append(data_x, a, axis=0)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [21]:
print data_x.shape

(40000, 100, 3)


In [20]:
ctr=0
with open(cloud) as f:
	for line in f:
		a,b = parse_line(line)
		if a.shape[0]>=100:
			ctr+=1
			print ctr
			if ctr>10000:
				break
			a = a[:100,:]
			a[-1,2] = 1.0
			a = a.reshape(1,100,3)
			data_x = np.append(data_x, a, axis=0)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [22]:
data_y = np.zeros((1,40000))
print data_y.shape
data_y[:,:10000] = 0
data_y[:,10000:20000] = 1
data_y[:,20000:30000] = 2
data_y[:,30000:] = 3

(1, 40000)


In [23]:
from keras.utils import np_utils
data_y = data_y.reshape(40000,)
data_y = np_utils.to_categorical(data_y)
print data_y.shape

(40000, 4)


In [0]:
model = Sequential()
model.add(Convolution1D(128,(6),activation='relu', input_shape=(100,3)))
model.add(Convolution1D(64,(3),activation='relu'))
model.add(Convolution1D(1, (1), activation='relu'))
model.add(LSTM(128, return_sequences=True))
# model.add(Dropout(0.5))
model.add(LSTM(128, return_sequences=True))
# model.add(Dropout(0.5))
model.add(LSTM(128, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(4))
model.add(Activation('softmax'))

In [25]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 95, 128)           2432      
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 93, 64)            24640     
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 93, 1)             65        
_________________________________________________________________
lstm_1 (LSTM)                (None, 93, 128)           66560     
_________________________________________________________________
lstm_2 (LSTM)                (None, 93, 128)           131584    
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
__________

In [0]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
checkpoint = [ModelCheckpoint(filepath='/content/drive/My Drive/models.hdf5')]

In [29]:
history = model.fit(data_x, # Features
						data_y, # Target vector
						epochs=30, # Number of epochs
						callbacks=checkpoint, # Checkpoint
						batch_size=100, # Number of observations per batch
						shuffle = True,
						validation_split=0.10)

Train on 36000 samples, validate on 4000 samples
Epoch 1/30
36000/36000 [==============================] - 274s 8ms/step - loss: 1.1357 - acc: 0.4952 - val_loss: 1.1875 - val_acc: 0.8915
Epoch 2/30
36000/36000 [==============================] - 274s 8ms/step - loss: 1.0881 - acc: 0.5295 - val_loss: 1.1793 - val_acc: 0.8930
Epoch 3/30
36000/36000 [==============================] - 274s 8ms/step - loss: 1.0349 - acc: 0.5609 - val_loss: 1.1151 - val_acc: 0.8913
Epoch 4/30
36000/36000 [==============================] - 276s 8ms/step - loss: 1.0078 - acc: 0.5761 - val_loss: 1.2101 - val_acc: 0.5900
Epoch 5/30
36000/36000 [==============================] - 275s 8ms/step - loss: 0.9763 - acc: 0.5954 - val_loss: 1.2205 - val_acc: 0.6613
Epoch 6/30
36000/36000 [==============================] - 271s 8ms/step - loss: 0.9520 - acc: 0.6074 - val_loss: 1.0381 - val_acc: 0.7388
Epoch 7/30
36000/36000 [==============================] - 272s 8ms/step - loss: 0.9225 - acc: 0.6245 - val_loss: 1.0566 - v

In [30]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
model.save('/content/drive/My Drive/Quick_Draw_LSTM.h5')